# Keras and TensorFlow.js Interoperability

Ralph Schlosser, 17/12/2018, <http://warpbreaks.io>

Git: https://github.com/bwv988/keras-tensorflowjs-tests/


## Introduction
With the release of TensorFlow.js earlier in 2018 (https://js.tensorflow.org/), it is now possible to train and run Machine Learning models directly in a web browser and NodeJS-based applications. TensorFlow.js implements much of TensorFlow's proven API and thus opens up interesting possibilities to augment web applications with Machine Learning capabilities.

This short notebook demonstrates a very simple workflow:

* Train a model using Keras and Python.
* Save the trained model.
* Load the finished model into TensorFlow.js / Javascript and make predictions.

As always, a picture says more than a thousand words, so below is a very detailed and highly technical diagram of this workflow:

![Workflow](https://github.com/bwv988/keras-tensorflowjs-tests/blob/master/assets/workflow.PNG?raw=1)

To demonstrate the last part, we'll use a special cell in Google Colab to load and process Javascript.


## Demo: Train offline, predict online
Many real-world ML problems require complex neural network architectures, the training of which, while now feasible, wouldn't be very practical to do in a browser. This is especially valid for mobile applications.

A far more common use case, however, would be to fetch a pre-trained model which was build e.g. on a GPU cluster in order to then make predictions. 

In the case of a neural networks, training the model entails establishing the weights, which is typically orders of magnitudes more expensive in terms of CPU compared to just applying the trained weights to make predictions, based on new data.

## Part 1: Offline Model Training

For this demonstration, I'll use the classical **Iris flower** dataset. The goal here is to predict the type (class) of flower based on four different measurements in cm taken from 150 flowers. The dataset is build into the `sklearn` package.

### Data loading and preparation

In [0]:
from sklearn import datasets
from sklearn import preprocessing
from keras.utils import to_categorical


# Load and transform the data.
iris = datasets.load_iris()
iris_target_onehot = to_categorical(iris.target)
X = preprocessing.scale(iris.data)
Y = iris_target_onehot

### Defining the model

For this multi-class classification problem, we'll be setting up a simple feed-forward neural network. 

**Note**: In this very simple case it would be perfectly OK to define and train the model in the browser. But I wanted something that runs quickly and I wouldn't have to wait for an hour for the results to be complete. :)

In [0]:
def build_iris_model(num_hidden_layers = 1, 
                     x_dim = 4, 
                     num_neurons = 64, 
                     my_optimizer = "adam"):
    """
    This function returns a feed-forward neural network, as
    defined in the parameters.
    """
    model = Sequential()
    model.add(Dense(num_neurons, 
                 input_dim = x_dim, 
                 activation = "relu"))
    
    # Add specified number of hidden layers.
    for i in range(num_hidden_layers):        
        model.add(Dense(num_neurons, 
                        activation = "relu"))
    
    model.add(Dense(3, activation = "softmax"))
    
    # Compile the model according to the selected optimizer function.
    # Using categorical cross entropy because we are classifying.
    model.compile(loss = "categorical_crossentropy", 
               optimizer = my_optimizer, 
               metrics = ["accuracy"])
    return model

### Training the model

Next, we'll fit the model to the training data (60% of the data).

**Note**: The data splitting isn't really necessary here, as I'm not doing any validation after.

Just leaving it in for the inevitable copy-and-paste :)

In [0]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size = 0.4, 
                                                    random_state = 0)

# Train the model.
model = build_iris_model()
model.fit(X_train, 
          Y_train, 
          validation_split = 0.25, 
          epochs = 150, 
          batch_size = 10, 
          verbose = 0)

### Testing the model

We can now convince ourselves that the model predicts well on the training data, as expected.

In [0]:
import numpy as np

class1_sample = np.reshape(X_train[0, :], (1, 4))
print("Sample data for class 1: " + str(class1_sample))
class0_sample = np.reshape(X_train[50, :], (1, 4))
print("Sample data for class 0: " + str(class0_sample))


print("\n\nThe first sample should yield class 1: " + str(model.predict_classes(class1_sample)))
print("The second sample should yield class 0: " + str(model.predict_classes(class0_sample)))

Sample data for class 1: [[0.18982966 0.78880759 0.42173371 0.52740629]]
Sample data for class 0: [[-1.02184904  0.32841405 -1.45390138 -1.3154443 ]]


The first sample should yield class 1: [1]
The second sample should yield class 0: [0]


### Saving the model

So far so good, nothing particularly fancy happening yet. Now we need to save the model.

Out of the box, Keras supports the HDF5 model format. However, for TensorFlow.js we need to convert into TensorFlow's Layers format. This can be achieved with a utility function within the `tensorflowjs` package.

**Note** For the next line to work, the `tensorflowjs` package needs to be installed. The procedure depends on where you're running this notebook.

* Anaconda / Google Collaboratory: `pip install tensorflowjs`
* Kaggle: Add a custom package.


In [0]:
!pip install tensorflowjs

    100% |████████████████████████████████| 61kB 4.6MB/s 
    100% |████████████████████████████████| 13.9MB 1.2MB/s 
    100% |████████████████████████████████| 307kB 23.4MB/s 
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.6 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.5 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
cufflinks 0.14.6 has requirement plotly>=3.0.0, but you'll have plotly 1.12.12 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: tensorflow-hub 0.2.0
    Uninstalling tensorflow-hub-0.2.0:
      Successfully uninstalled tensorflow-hub-0.2.0
  Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.

In [0]:
import tensorflowjs as tfjs

# Modify as needed.
TFJS_MODEL_PATH = "/tmp/irismodel-tfjs"

# Save the model in TF's Layer format.
tfjs.converters.save_keras_model(model, TFJS_MODEL_PATH)

In [0]:
!ls -la /tmp/irismodel-tfjs

total 32
drwxr-xr-x 2 root root  4096 Dec 18 15:14 .
drwxrwxrwt 1 root root  4096 Dec 18 15:14 ..
-rw-r--r-- 1 root root 18700 Dec 18 15:14 group1-shard1of1
-rw-r--r-- 1 root root  2355 Dec 18 15:14 model.json


In production, there would be a step to publish this model to a cloud storage provider, like Amazon S3, or Google Cloud Storage. But as this is just a demo, I'm skipping this step.


## Part 2: Online Model Loading and Predictions

Now, we'll load the pre-trained model into Javascript and make predictions.

I'm repeating the validation step from before, only this time the prediction code is in Javascript and we load the model from a URL using TensorFlow.js.

A few notes on the below code cell:

* Unlike in vanilla Jupyte, Google Colaboratory doesn't support `require.js` in the the `%%javascript` "cell magic". Therefore we need to fetch it first. 

* Another difference is in how to add HTML to the cell output.

* Obviously for a real application, we'd separate out the code bits into a module and use Yarn or NPM to load the TensorFlow.js library.

* The computation results are just the raw class probabilities. The highest probability corresponds to the predicted class.

In [0]:
def load_and_configure_js_components():
  """Helper function needed in Google Colaboratory."""
  import IPython
  display(IPython.core.display.HTML('''
        <!-- Load require.js from a CDN. -->
        <script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.6/require.min.js"></script>
        <script>
          requirejs.config({
            paths: { 
              'tf': ['//cdn.jsdelivr.net/npm/@tensorflow/tfjs@0.14.1/dist/tf.min'], 
            },
          });
          
          // Load and use TensorFlow.js.
          require(['tf'], function(tf) {
              async function load_and_predict() {
                  // This is where we are loading the model from. 
                  // Could also be Google cloud storage or an Amazon S3 bucket.
                  const MODEL_URL = "https://raw.githubusercontent.com/bwv988/keras-tensorflowjs-tests/master/model/model.json";

                  console.log("Loading pre-trained model...")
                  const model = await tf.loadModel(MODEL_URL);
                  console.log("Model successfully loaded.")

                  // This the first training example which is of class 1.
                  const class1_test = tf.tensor([
                    [0.18982966, 0.80065426, 0.42156442, 0.52764485]
                  ]);

                  // This is another training example of class 0.
                  const class0_test = tf.tensor([
                    [-1.02184904,  0.33784833, -1.45500381, -1.31297673]
                  ]);

                  // Contruct the output.
                  // FIXME: This is a bit crude ;)
                  const p1 = model.predict(class1_test);
                  const p2 = model.predict(class0_test);

                  let res_str = `
                  <strong>Predictions:</strong>
                  <br/>
                  <br/>
                  Predicted class probabilities for class 1 training example:
                  `;

                  res_str += p1;

                  res_str += `
                  <br/>
                  Predicted class probabilities for class 0 training example:
                  `
                  res_str += p2;
                  
                  // Append output to div.
                  div = document.getElementById("my_output");
                  div.innerHTML = res_str;                      
              }

              load_and_predict();
              return {}
              });
        </script>
        
        <!-- Here is where we will apend the output from Javascript.-->
        <div id="my_output"/>

    '''))
  
load_and_configure_js_components()

In [0]:
%%javascript
require(['tf'], function(tf) {
    async function load_and_predict() {
        // This is where we are loading the model from. 
        // Could also be Google cloud storage or an Amazon S3 bucket.
        const MODEL_URL = "https://raw.githubusercontent.com/bwv988/keras-tensorflowjs-tests/master/model/model.json";
  
        console.log("Loading pre-trained model...")
        const model = await tf.loadModel(MODEL_URL);
        console.log("Model successfully loaded.")
        
        // This the first training example which is of class 1.
        const class1_test = tf.tensor([
          [0.18982966, 0.80065426, 0.42156442, 0.52764485]
        ]);
        
        // This is another training example of class 0.
        const class0_test = tf.tensor([
          [-1.02184904,  0.33784833, -1.45500381, -1.31297673]
        ]);
        
        // Contruct the output.
        // FIXME: This is a bit crude ;)
        const p1 = model.predict(class1_test);
        const p2 = model.predict(class0_test);
        
        let res_str = `
        <strong>Predictions:</strong>
        <br/>
        <br/>
        Predicted class probabilities for class 1 training example:
        `;
        
        res_str += p1;
        
        res_str += `
        <br/>
        Predicted class probabilities for class 0 training example:
        `
        res_str += p2;
        
        element.html(res_str);
        
}

load_and_predict();
    return {}
});

<IPython.core.display.Javascript object>

## Summary

As we can see, it's fairly simple to load pre-trained models in TensorFlow.js. This also shows that Keras, Tensorflow, Python and Javascript play a long quiet nicely and enable ML developers to seamlessly connect different execution domains.

## References and Links

* TensorFlow.js homepage: https://js.tensorflow.org/
* Introduction post on Medium: https://medium.com/tensorflow/introducing-tensorflow-js-machine-learning-in-javascript-bf3eab376db
* Jupyter embracing modern web standards: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/JavaScript%20Notebook%20Extensions.html